<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/SentimentClassification_Fnet_01_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requ

In [2]:
!pip install keras-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 28.0 MB/s eta 0:00:00


# Fnet is a fast anmd small versions of transformer based models where Fnet uses fourier transform instead of using self-attention mechanism but the purpose is same here also

In [3]:
import keras
import keras_nlp

import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers, losses, optimizers

In [4]:
columns = ["id", "country", "Label", "Text"]

tweets_data = pd.read_csv("twitter_training.csv", names = columns)
tweets_data = tweets_data.sample(frac = 0.001)
print(tweets_data.shape)

tweets_data.sample(5)

(75, 4)


,id,country,Label,Text
10738,13046,Xbox(Xseries),Negative,Just a letting you know I'll be buying on a ps...
25889,843,AssassinsCreed,Positive,Another cool weapon piece for Assassins Creed ...
44755,11686,Verizon,Neutral,"Tricia B., a human resources consultant at Ver..."
70299,10844,TomClancysGhostRecon,Negative,@GhostRecon There has been a sound issue since...
39809,1234,Battlefield,Neutral,DICE announced today that Battlefield V will o...


In [5]:
tweets_data = tweets_data.drop(columns = ["id", "country"])

tweets_data.dropna(inplace = True, axis = 0)

tweets_data = tweets_data.drop_duplicates()

tweets_data.shape

(75, 2)

In [6]:
tweets_data["Label"] = tweets_data["Label"].replace({"Negative": 0, "Neutral": 1, "Positive": 2, "Irrelevant": 3})

tweets_data.sample(5)

<ipython-input-6-f68799292937>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tweets_data["Label"] = tweets_data["Label"].replace({"Negative": 0, "Neutral": 1, "Positive": 2, "Irrelevant": 3})


,Label,Text
25281,0,A Google Search bug wreaked havoc on the searc...
21149,3,s1mple - NEW BEST Online:GO Player Of The Box!...
55006,0,In this it is the WORST @CallofDuty the EVER!!
7674,2,Time for some more boops and kills in @PlayOve...
14559,3,Nice list


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(
    tweets_data, test_size = 0.2, stratify = tweets_data["Label"], random_state = 123)
X_train, X_val = train_test_split(
    X_train, test_size = 0.1, stratify = X_train["Label"], random_state = 123)

X_train.shape, X_val.shape, X_test.shape

((54, 2), (6, 2), (15, 2))

In [8]:
BATCH_SIZE = 128

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train["Text"].values, X_train["Label"].values)).shuffle(10000).batch(batch_size = BATCH_SIZE)

val_ds = tf.data.Dataset.from_tensor_slices(
    (X_val["Text"].values, X_val["Label"].values)).batch(batch_size = BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices(
    (X_test["Text"].values, X_test["Label"].values)).batch(batch_size = BATCH_SIZE)

In [9]:
train_ds = train_ds.map(lambda x, y: (tf.strings.lower(x), y))

val_ds = val_ds.map(lambda x, y: (tf.strings.lower(x), y))

test_ds = test_ds.map(lambda x, y: (tf.strings.lower(x), y))

In [10]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch[i])
        print(label_batch[i])

tf.Tensor(b'@ cfduty @ blizzard _ ent seriously.', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'time for some more boops and kills in @playoverwatch on @twitch. lets get that new torb skin . .  . . twitch.tv/nakubael https://t.co/nj1kwfi8m6', shape=(), dtype=string)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(b'a google search bug wreaked havoc on the search results of weekend dlvr.it/rdr4fb https://t.co/7l91i6qzkn]', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [11]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'its almost june and @rockstargames red dead redemption 2 on pc still crashes without even a acknowledgement from the developers. fix your fucking game dickheads.'
0
b'xbox fantasy x gets so hot but claims users have washed their hand metro.co.uk/2020/10/06/xbo... via the'
1
b"was going to make you a tweet about today how lost among us has made me all feel totally alienated... from my friends because i don't enjoy it but i remembered people have moved on and from the games i helped enjoy years ago with ahaha fuck if anyone plays rainbow 6 ps or cs : go in europe hit me up"
2


In [12]:
# keras-nlp uses wordpiece tokenization which is a subword tokenization
def train_word_piece(ds, vocab_size, reserved_tokens):

    word_piece_ds = ds.unbatch().map(lambda x, y: x)

    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size = vocab_size,
        reserved_tokens = reserved_tokens,
    )

    return vocab


In [13]:
vocab_size = 10000

reserved_tokens = ["[PAD]", "[UNK]"] #Pad token , unknown token for  OOV tokens

# train_sentences = [element[0] for element in train_ds]
vocab = train_word_piece(train_ds, vocab_size, reserved_tokens)

In [14]:
len(vocab)

143

In [15]:
vocab

['[PAD]',
 '[UNK]',
 '!',
 '"',
 '$',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 '@',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '’',
 '…',
 '🤣',
 '##s',
 '##e',
 '##d',
 '##ing',
 '##y',
 '##t',
 '##o',
 'the',
 '##ed',
 '##r',
 '##on',
 '##er',
 '##l',
 '##n',
 '##le',
 '##m',
 '##st',
 '##ut',
 'and',
 '##es',
 'it',
 'of',
 'for',
 'to',
 '##re',
 '##ch',
 '##h',
 'in',
 'my',
 'on',
 'you',
 '##a',
 '##ll',
 '##om',
 '##ot',
 '##ow',
 '##ck',
 '##k',
 '##ve',
 '##hat',
 '##an',
 '##g',
 '##is',
 '##et',
 '##se',
 '##!',
 '##"',
 '##$',
 '##&',
 "##'",
 '##(',
 '##)',
 '##,',
 '##-',
 '##.',
 '##/',
 '##0',
 '##1',
 '##2',
 '##3',
 '##4',
 '##5',
 '##6',
 '##7',
 '##8',
 '##9',
 '##:',
 '##?',
 '##@',
 '##]',
 '##_',
 '##b',
 '##c',
 '##f',
 '##i',
 '##j',
 '##p',
 '##q',
 '##u',
 

In [16]:
max_sequence_length = 64 #all the documents will be padded to 64 or truncated to 64

tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = vocab,
    lowercase = False,
    sequence_length = max_sequence_length,
)

In [17]:
input_sentence_ex = train_ds.take(1).get_single_element()[0][1]
input_tokens_ex = tokenizer(input_sentence_ex)

print("Sentence: ", input_sentence_ex)
print("Tokens: ", input_tokens_ex)
print("Recovered text after detokenizing: ", tokenizer.detokenize(input_tokens_ex))

Sentence:  tf.Tensor(b"its it's ain't much. but i still happy i got there. redd.it/hcte5f  ", shape=(), dtype=string)
Tokens:  tf.Tensor(
[ 77  57  77   6  46  28 131  70   6  47  40 135  82  11  29  74  36  46
  62 131  89  35  88 133 133  61  36  34  91  64  81  11  45  65  59  11
  77  12  35 129  62  58 118 130   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0], shape=(64,), dtype=int32)
Recovered text after detokenizing:  its it ' s ain ' t much . but i still happy i got there . redd . it / hcte5f [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [18]:
def format_dataset(sentence, label):

    sentence = tokenizer(sentence)

    return ({"input_ids": sentence}, label)


def make_dataset(dataset):

    dataset = dataset.map(format_dataset, num_parallel_calls = tf.data.AUTOTUNE)

    return dataset.shuffle(10000).prefetch(512).cache()


train_ds = make_dataset(train_ds)
val_ds = make_dataset(val_ds)
test_ds = make_dataset(test_ds)

In [19]:
epochs = 5

embedding_dim = 128
intermediate_dim = 256

In [20]:
input_ids = keras.Input(shape = (None,), dtype = "int64", name = "input_ids")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size = vocab_size,
    sequence_length = max_sequence_length,
    embedding_dim = embedding_dim,
    mask_zero = True,
)(input_ids)

x = keras_nlp.layers.FNetEncoder(intermediate_dim = intermediate_dim)(inputs = x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim = intermediate_dim)(inputs = x)
x = keras_nlp.layers.FNetEncoder(intermediate_dim = intermediate_dim)(inputs = x)

x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)

outputs = keras.layers.Dense(4, activation = "softmax")(x)

fnet_classifier = keras.Model(input_ids, outputs, name = "fnet_classifier")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'f_net_encoder' (of type FNetEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [21]:
fnet_classifier.summary()

fnet_classifier.compile(
    optimizer = optimizers.Adam(learning_rate = 0.001),
    loss = losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ["accuracy"],
)

fnet_classifier.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "fnet_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)               │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 128)           │       1,288,192 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ f_net_encoder (FNetEncoder)          │ (None, None, 128)           │          66,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ f_net_encoder_1 (FNetEncoder)        │ (None, None, 128)           │          66,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ f_net_encoder_2 (FNetEncoder)        │ (None, None, 128)           │          66,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,488,004 (5.68 MB)

 Trainable params: 1,488,004 (5.68 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: input_ids. Received: the structure of inputs={'input_ids': '*'}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'position_embedding' (of type PositionEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.2037 - loss: 1.4685 - val_accuracy: 0.3333 - val_loss: 1.4264
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.3333 - loss: 1.3453 - val_accuracy: 0.3333 - val_loss: 1.4162
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.4259 - loss: 1.3330 - val_accuracy: 0.3333 - val_loss: 1.4035
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.5741 - loss: 1.2457 - val_accuracy: 0.3333 - val_loss: 1.3690
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7037 - loss: 1.1702 - val_accuracy: 0.3333 - val_loss: 1.3584


In [22]:
loss, accuracy = fnet_classifier.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - accuracy: 0.2667 - loss: 1.4255
Loss:  1.4255081415176392
Accuracy:  0.2666666805744171
